<a href="https://colab.research.google.com/github/ballban/cryptopals/blob/main/cryptopals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set_1

Challenge 1

In [1]:
import base64

text = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
ans = 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

baseInt= int(text, 16)
print(baseInt)

bytes_text = bytes.fromhex(text)
print(bytes_text)

result = base64.b64encode(bytes_text)
print(result)

11259432467145572969189485457381052543241507215288737798329079056359121649591228422793827173000297562297701340508013
b"I'm killing your brain like a poisonous mushroom"
b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'


Challenge 2

In [2]:
buffer1 = '1c0111001f010100061a024b53535009181c'
buffer2 = '686974207468652062756c6c277320657965'
ans = '746865206b696420646f6e277420706c6179'

hex1 = bytes.fromhex(buffer1)
hex2 = bytes.fromhex(buffer2)
print(hex1)
print(hex2)

result = ''
for i in range(len(hex1)):
    result += str(hex(hex1[i] ^ hex2[i]))[2:]

print(result)
print(result == ans)

b'\x1c\x01\x11\x00\x1f\x01\x01\x00\x06\x1a\x02KSSP\t\x18\x1c'
b"hit the bull's eye"
746865206b696420646f6e277420706c6179
True


Challenge 3

In [3]:
from collections import defaultdict
def GetFrequencyDic(text):
  fre_bytes = defaultdict(int)
  for i in range(len(text)):
    x = text[i]
    fre_bytes[x] += 1
  return sorted(fre_bytes.items(), key=lambda x:x[1], reverse=True)

In [4]:
import string

score_dic = {"E":11.16,"A":8.50,"R":7.58,"I":7.54,"O":7.16,"T":6.95,"N":6.65,"S":5.74,"L":5.49,"C":4.54,"U":3.63,"D":3.38,"P":3.17,"M":3.01,"H":3.00,"G":2.47,"B":2.07,"F":1.81,"Y":1.78,"W":1.29,"K":1.10,"V":1.01,"X":0.29,"Z":0.27,"J":0.20,"Q":0.20,"e":11.16,"a":8.50,"r":7.58,"i":7.54,"o":7.16,"t":6.95,"n":6.65,"s":5.74,"l":5.49,"c":4.54,"u":3.63,"d":3.38,"p":3.17,"m":3.01,"h":3.00,"g":2.47,"b":2.07,"f":1.81,"y":1.78,"w":1.29,"k":1.10,"v":1.01,"x":0.29,"z":0.27,"j":0.20,"q":0.20," ":18}
def GetScore(text):
  score = 0
  for key, value in GetFrequencyDic(text):
    if key in score_dic:
      score += score_dic[key] * value
    elif key not in string.printable:
      score = 0
      break
  return score

In [5]:
def TryDecrypt(text):
  if(type(text) == str):
    text_bytes = bytes.fromhex(text)
  else:
    text_bytes = text
  #print(text_bytes)

  keys = range(0, 127)
  scores = defaultdict(int)
  for key in keys:
    decrypt_text = ''
    for i in range(len(text_bytes)):
      x = text_bytes[i]
      decrypt_text += chr(x ^ key)
    scores[key] = GetScore(decrypt_text)
  scores = sorted(scores.items(), key=lambda x:x[1], reverse=True)
  #print(scores[:10])

  true_key = scores[0][0]

  result = ''
  for i in range(len(text_bytes)):
      x = text_bytes[i]
      result += chr(x ^ true_key)
  return scores[0][1], result, true_key

In [6]:
text = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'

print(TryDecrypt(text))

(253.57999999999998, "Cooking MC's like a pound of bacon", 88)


Challenge 4

In [7]:
import requests
def GetTxt(url):
  f = requests.get(url).content.decode('utf-8')
  text_list = f.split('\n')
  #f = [bytes.fromhex(x) for x in f]
  #print(text_list)
  return text_list

text_list = GetTxt("https://cryptopals.com/static/challenge-data/4.txt")

In [8]:
decrypt_list = []
for text in text_list:
  decrypt_list.append(TryDecrypt(text))
result = sorted(decrypt_list, key=lambda x:x[0], reverse=True)
print(*result[:10], sep="\n")

(219.54000000000002, 'Now that the party is jumping\n', 53)
(113.88999999999997, '[$LrLi;hsDbv!5 j$OaEfG=>rKDl=S', 109)
(113.14999999999999, 'wpUs{gnns F8b-aT2*zRAh{Gj>xuED', 98)
(109.14, "iu`'ern.&nZeM,MjT\x0b:FD~i5+OyEmL", 113)
(105.78999999999999, 'Sw6XSo$^uWyP$C{iUCns3e?E` /]GZ', 108)
(90.35000000000001, 'T2XNi-]1rTIYbP/>2`o%%]T5JonmqA', 99)
(70.12000000000002, ']$4OFs>\\YQ#\nK9y?IT4PeNWTxFqc0\r', 103)
(70.01000000000002, '\\`)]^o}me|~#Am~t[dbF.tq\x0bmAWM|~', 114)
(0, "\x0e6GèY-5QJ\x08\x12CX%6í=æs@Y\x00\x1e?S\\æ'\x102", 0)
(0, '3K\x04\x1dá$÷<\x18\x01\x1aPæ\x08\tzÃ\x08ìîP\x137ì>\x10\x08T \x1d', 0)


Challenge 5

In [9]:
text = '''Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal'''

key = 'ICE'

def Encrypt(text, key):
  key = key * int(len(text) / len(key) + 1)
  result = b''
  for i in range(len(text)):
    result += (ord(text[i]) ^ ord(key[i])).to_bytes(1, 'big')
  return result

print(Encrypt(text, key).hex())


0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f


Challenge 6

In [10]:
text_list = GetTxt("https://cryptopals.com/static/challenge-data/6.txt")

In [11]:
text = "".join(text_list)
text = base64.b64decode(text)

print(text)
print(len(text))

b'\x1dB\x1fM\x0b\x0f\x02\x1fO\x13N<\x1aie\x1fI\x1c\x0eN\x13\x01\x0b\x07N\x1b\x01\x16E6\x00\x1e\x01Id T\x1d\x1dC3SNeR\x06\x00GT\x1c\rEM\x07\x04\x0cS\x12<\x0c\x1e\x08I\x1a\t\x11O\x14L!\x1aG+\x00\x05\x1dGY\x11\x04\t\x00d&\x07S\x007\x16\x06\x0c\x1a\x17A\x1d\x01RT0_\x00 \x13\n\x05GO\x12H\x08ENe>\x16\t8E\x06\x05\x08\x1aF\x07O\x1fYx~jb6\x0c\x1d\x0fA\rH\x06U\x1a\x1b\x00\x1dBt\x04\x1e\x01I\x1a\t\x11\x02Rz\x7fI\x00H:\x00\x1a\x13I\x1aOEH\x0f\x1d\rS\x04:\x01R\x19\x01\x0bA\x13\x06\x00L1_Sb\x15\x06\x07\t\x07T\x0b\x17A\x14\x16Iy35\x0b\x1b\x01\x05\x0fF\x07O\x1dNxNH\'R\x04\x07\x0cEXH\x08A\x00O T\x08t\x0b\x1d\x19I\x02\x00\x0e\x16\\\x00R0ie\x1fI\x02\x02T\x00\x01\x0b\x07N\x02\x10S\x01&\x10\x15M\x02\x07\x02\x1fO\x1bNx0i6R\n\x01\tT\x06\x07\tSN\x02\x10S\x08;\x10\x06\x05I\x0f\x0f\x10O;\x00:_G+\x1cId3OT\x02\x10S\x1aO\x05\x16\x11t\x0c\x06M\x0f\x02\x0e\x03CRL=N\x00/\x0bI\r\x08N\x17\r\x15T\x1dO\x0e\x1cE^(\x0bM\x19\x01\x12\x07\nUSxNOb\x06\x01\x0bGS\x1d\x0c\x00\x00\x17\n\x05\x1f\x0c:B^M.\x01A"\x0e\x1cI4VAb5\x06OG*~

In [12]:
def HammingDistance(text1, text2):
  # text1 = bytes(text1, 'utf-8')
  # text2 = bytes(text2, 'utf-8')
  return sum([bin(x[0] ^ x[1]).count("1") for x in zip(text1, text2)])

In [13]:
test_Normalized = defaultdict(list)
Normalized_list = dict()
for keysize in range(2, 41):
  block_list = []
  for i in range(int(len(text)/keysize) + 1):
    block_list += [text[keysize*i:keysize*(i+1)]]
  
  distance_list = []
  for i in range(len(block_list) - 1):
    distance_list += [HammingDistance(block_list[i],block_list[i+1])]
  Normalized_list[keysize] = sum(distance_list)/len(distance_list)/keysize

  
  for i in range(2, len(distance_list) + 1):
    test_distance_list = distance_list[:i]
    test_Normalized[i] += [(keysize, sum(test_distance_list)/len(test_distance_list)/keysize)]

max_chunk = int(len(text) / 40) + 1
for chunk, normalized_list in test_Normalized.items():
  if chunk > max_chunk:
    break
  print(f"for using {chunk} chunk, best 5 scores are : {sorted(normalized_list, key=lambda x:x[1])[:5]}")

print(*sorted(Normalized_list.items(), key=lambda x:x[1])[:10], sep='\n')
candidate_keysize_list = [x[0] for x in sorted(Normalized_list.items(), key=lambda x:x[1])[:10]]

transpose_block_dict = dict()
for keysize in candidate_keysize_list:
  block_list = []
  for i in range(keysize):
    block_list += [text[i::keysize]]
  transpose_block_dict[keysize] = block_list

print(transpose_block_dict)
for keysize, transpose_block_list in transpose_block_dict.items():
  decrypt_list = []
  for index, transpose_block in enumerate(transpose_block_list):
    score, _, key = TryDecrypt(transpose_block)
    decrypt_list.append((index, (score, key)))
  #print(decrypt_list)

  key = "".join([chr(x[1][1]) for x in decrypt_list])
  if all([x in string.printable for x in key]):
    print(f"key: {key}")

for using 2 chunk, best 5 scores are : [(2, 2.25), (5, 2.3), (3, 2.3333333333333335), (18, 2.8333333333333335), (20, 2.85)]
for using 3 chunk, best 5 scores are : [(2, 2.0), (3, 2.6666666666666665), (29, 2.793103448275862), (5, 2.8), (18, 2.9444444444444446)]
for using 4 chunk, best 5 scores are : [(2, 2.25), (5, 2.6), (29, 2.8189655172413794), (3, 2.8333333333333335), (18, 3.0555555555555554)]
for using 5 chunk, best 5 scores are : [(2, 2.4), (5, 2.52), (3, 2.8000000000000003), (29, 2.8344827586206898), (31, 3.0258064516129033)]
for using 6 chunk, best 5 scores are : [(2, 2.5833333333333335), (5, 2.7), (3, 2.8333333333333335), (29, 2.8563218390804597), (18, 3.0185185185185186)]
for using 7 chunk, best 5 scores are : [(5, 2.7142857142857144), (29, 2.8226600985221677), (3, 2.857142857142857), (15, 2.99047619047619), (18, 3.0238095238095237)]
for using 8 chunk, best 5 scores are : [(5, 2.775), (29, 2.8189655172413794), (3, 2.8333333333333335), (2, 3.0), (15, 3.0833333333333335)]
for usin

Chellange 7

In [14]:
text_list = GetTxt("https://cryptopals.com/static/challenge-data/7.txt")
text = ''.join(text_list)
text = base64.b64decode(text)
key = b'YELLOW SUBMARINE'

In [15]:
%pip install PyCryptoDome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.5 MB/s eta 0:00:00


In [16]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

def decrypt_ECB(text: bytes, key) -> bytes:
  cipher = AES.new(key, AES.MODE_ECB)
  return cipher.decrypt(text)

def encrypt_ECB(text: bytes, key, block_size: int) -> bytes:
  cipher = AES.new(key, AES.MODE_ECB)
  encrypted_bytes = b''
  for i in range(len(text)//block_size + 1):
    chunk = text[i*block_size:(i+1)*block_size]
    if(len(chunk) == 0):
      return encrypted_bytes
    pad_count = block_size - len(chunk)
    pad_chunk = chunk + b'\x00' * pad_count
    encrypted_bytes += cipher.encrypt(pad_chunk)
  return encrypted_bytes


In [17]:
print(decrypt_ECB(text, key))

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

Challenge 8

In [18]:
text_list = GetTxt("https://cryptopals.com/static/challenge-data/8.txt")

In [19]:
print(len(text_list))

205


In [20]:
target_index = 0
for index, text in enumerate(text_list):
  text = bytes.fromhex(text)
  if len(text) < 1:
    continue
  for i in range(10):
    for j in range(i + 1, 10):
      if text[i*16: (i+1)*16] == text[j*16: (j+1)*16]:
        print(index, text)
        target_index = index
        break

print(text_list[target_index])

132 b'\xd8\x80a\x97@\xa8\xa1\x9bx@\xa8\xa3\x1c\x81\n=\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xe2\xdd\x05/kd\x1d\xbf\x9d\x11\xb04\x85B\xbbW\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x94u\xc9\xdf\xdb\xc1\xd4e\x97\x94\x9d\x9c~\x82\xbfZ\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x97\xa9>\xab\x8dj\xec\xd5fH\x91Tx\x9ak\x03\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xd4\x03\x18\x0c\x98\xc8\xf6\xdb\x1f*?\x9c@@\xde\xb0\xabQ\xb2\x993\xf2\xc1#\xc5\x83\x86\xb0o\xba\x18j'
132 b'\xd8\x80a\x97@\xa8\xa1\x9bx@\xa8\xa3\x1c\x81\n=\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xe2\xdd\x05/kd\x1d\xbf\x9d\x11\xb04\x85B\xbbW\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x94u\xc9\xdf\xdb\xc1\xd4e\x97\x94\x9d\x9c~\x82\xbfZ\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x97\xa9>\xab\x8dj\xec\xd5fH\x91Tx\x9ak\x03\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xd4\x03\x18\x0c\x98\xc8\xf6\xdb\x1f*?\x9c@@\xde\xb0\xabQ\xb2\x993\xf2\xc1#\xc5\x83\x86\xb0o\xba\x18j'
132 b'\xd8\x80a\x97@\xa8\xa1\x

# Set_2

Challenge 9

In [21]:
def PKCS7(input_bytes, block_size):
  pad_length = block_size - len(input_bytes) % block_size
  return str.encode(input_bytes + chr(pad_length) * pad_length)

In [22]:
PKCS7("YELLOW SUBMARINE", 20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

Challenge 10

In [23]:
def encrypt_CBC(bytes_text, key, block_size, initialization_vector):
  block_list = [bytes_text[i * block_size: (i+1) * block_size] for i in range(int(len(bytes_text) / block_size) + 1)]
  
  pre_block = initialization_vector
  result = []
  for block in block_list:
    if len(block) == 0:
      break
    XOR_block = [b1 ^ b2 for b1, b2 in zip(pre_block, block)]
    #print(f'block: {block}')
    #print(f'XOR_block: {XOR_block}')
    encrypted_block = encrypt_ECB(bytes(XOR_block), key, block_size)
    #print(f'encrypted_block: {[x for x in encrypted_block]}')

    pre_block = encrypted_block
    result += encrypted_block
  return bytes(result)


def decrypt_CBC(bytes_text, key, block_size, initialization_vector):
  block_list = [bytes_text[i * block_size: (i+1) * block_size] for i in range(int(len(bytes_text) / block_size))]
  
  pre_block = initialization_vector
  result = []
  for block in block_list:
    decrypted_block = decrypt_ECB(block, key)
    plain_text = [b1 ^ b2 for b1, b2 in zip(pre_block, decrypted_block)]

    pre_block = block
    result += plain_text
  return bytes(result)

In [24]:
plain_text_list = GetTxt("https://cryptopals.com/static/challenge-data/10.txt")
plain_text = ''.join(text_list)
key = b'YELLOW SUBMARINE'
print(plain_text)

...


In [26]:
block_size = 16
initialization_vector = b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

print(decrypt_CBC(plain_text, key, block_size, initialization_vector).decode('utf-8'))

UnicodeDecodeError: ignored

Challenge 11

In [27]:
import random
import os

def encryption_oracle(plain_text: str, block_size: int) -> bytes:
  bytes_text = plain_text.encode()
  random_key = random_aes_key()
  
  append_count = random.randint(5, 10)
  bytes_text = os.urandom(append_count) + bytes_text + os.urandom(append_count)

  encryption_mode = random.randint(0,1)

  if encryption_mode == 0:
    # ECB
    encrypted_bytes = encrypt_ECB(bytes_text, random_key, block_size)
  else:
    # CBC
    encrypted_bytes = encrypt_CBC(bytes_text, random_key, block_size, os.urandom(16))

  return encrypted_bytes, random_key, 'ECB' if encryption_mode == 0 else 'CBC'

def random_aes_key() -> bytes:
  return os.urandom(16)

In [28]:
block_size = 16
for i in range(10):
  encrypted_bytes, key, encryption_mode = encryption_oracle('A'*256, block_size)

  target_index = -1
  max_chunk = int(len(encrypted_bytes)/block_size)
    
  for i in range(max_chunk - 1):
    for j in range(i + 1, max_chunk):
      if encrypted_bytes[i*block_size: (i+1)*block_size] == encrypted_bytes[j*block_size: (j+1)*block_size]:
        #print(encrypted_bytes)
        #print(encrypted_bytes[i*block_size: (i+1)*block_size], encrypted_bytes[j*block_size: (j+1)*block_size], i, j)
        target_index = i
        break
    if target_index != -1:
      break
  
  detected_mode = 'ECB' if target_index != -1 else 'CBC'
  print(f'{encryption_mode == detected_mode}, answer: {encryption_mode}, detected: {detected_mode}')

True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: ECB, detected: ECB
True, answer: CBC, detected: CBC
True, answer: ECB, detected: ECB
True, answer: CBC, detected: CBC


Challenge 12

In [121]:
import os
import random

def new_oracle(plain_text) -> bytes:
  footer_base64 = 'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK'
  footer_bytes = base64.b64decode(footer_base64)

  if isinstance(plain_text, str):
    bytes_text = random_prefix + plain_text.encode() + footer_bytes
  else:
    bytes_text = random_prefix + plain_text + footer_bytes

  # ECB
  #print("new_oracle", [bytes_text[x*block_size: (x+1)*block_size] for x in range(len(bytes_text))])
  encrypted_bytes = encrypt_ECB(bytes_text, random_key, block_size)

  return encrypted_bytes

In [30]:
# Step 1
def detect_block_size(plain_text):
  for i in range(10):
    new_text = 'A' * i + plain_text
    ECB_encrypted_text = new_oracle(new_text)
    print(ECB_encrypted_text)
  return block_size

# Step 2
def detect_ECB_mode(plain_text: str, block_size: int):
  target_index = -1
  encrypted_bytes = new_oracle('A'*256 + plain_text)
  max_chunk = len(encrypted_bytes)//block_size

  for i in range(max_chunk - 1):
    for j in range(i + 1, max_chunk):
      if encrypted_bytes[i*block_size: (i+1)*block_size] == encrypted_bytes[j*block_size: (j+1)*block_size]:
        target_index = i
        break
    if target_index != -1:
      break

  return 'ECB' if target_index != -1 else 'UNKNOWN'

# Step 3,4
def craft_input_dict(header: str):
  return {new_oracle(header + bytes([i]))[:block_size]: bytes([i]) for i in range(256)}

# Step 5, 6
def match_output():
  header = b'A' * (block_size - 1)
  result = header

  for x in range(10):
    for i in range(block_size):
      input_dict = craft_input_dict(header)
      key = new_oracle(b'A' * (block_size - 1 - i))[x*block_size:(x+1)*block_size]
      if key in input_dict:
        output = input_dict[key]
      else:
        return result[block_size - 1:]
      result += output
      header = result[-block_size+1:]
  return result[block_size - 1:]

In [31]:
block_size = 16
random_key = os.urandom(block_size)
random_bytes = os.urandom(random.randint(5, 10))

encrypted_bytes = new_oracle(plain_text)
print(encrypted_bytes)

# Step 1 Get block_size
#detected_block_size = detect_block_size(plain_text)
#print(f'block size detection:{detected_block_size==block_size} block_size:{block_size} detected_block_size:{detected_block_size}')
# Step 2 Detected ECB mode
detected_mode = detect_ECB_mode(plain_text, block_size)
print(f'detected_mod: {detected_mode}')
# Step 3,4 Craft input
# Step 5,6 match_output
match_output()

b't\x87f\x98\x96H\xa1R\xdf$\t\xc0em\xba\xb5\x90d\xad\xe3O\xb5\xf3iL\xe1\xceJ\x92\xa9\x19[\xd5\xbf 4-!\xfc\n\x06\xdbz\x90\x00\xe9\xdc\xb89\xfa\xd1\x91\x17@\xdfp\x05\x84C\xbc\x86\xba\xfe\xc2UT\x11\xd8?gj\xe7\x04\xda\x1c\xe8\xe3\x0f\x9b\xcb\xd0\x04D\xbb\x93T\x91\x01\x8b\xf1\r1Z\xf2\xba\x8e\xd3\xa7\xda\xe8\xda\xdd.7\x8c\x9e\xe6\xf0\x94\x82\x9bw\xd1\x13\x1c\xee2o.\xaa,\xb2\x1f\x0e\xd6d\x83\xf8\x80&\xe3\xf3m\xf0*{\x18\x99\xff\xd30%\xb1b\xc2D=\xe0\xc6\xfb(\xeb\xa7\x1c\xeb\xd3\x9b\x87:\xed\xd1\x026\xd9\xc1|P\x8f\xc4\xaaG\xe8\x86\x86H\x10\xa5\xf3\x04\x88\xf34w\xeeu\xe2\xa2\xa1\x08\xd0<\xd5\xcbw\x82n]\x81\x0f\tGX\xc4\x16S\x1c\xb6\xb7\x0b~\\\xa3Y\x98z\x10\xb4Rr\xc3\xe8\xc3\xdf\x90\xff\x06\x02\xca\x16vg\xd6\xe5\x80,\xe9\xb3\x8a\xa75\xef=>4\xd2\xa2\xee\xca\xf9\x8e\xa0[\\\xad5\x0bQQ\xc9\x10A\x05$=\xb4\xb2\xe1\xa2J\xc8\xb8\xfd\xf7\xd9l1X\xea\xbc\x170]y\x02\x1a\xb0\xb3z\xa6\xa2\xc7\x0c\x82\xf8\xb7N\xa60\xd0A\xdf\x15\x97\xb8\xfdlo\xcb.\xf2\xd3I\xb9\xcdP\xb3\xd8\xd21l\xf2*1\xcc\xd0!\x19P\x98\xde\xfb#\x1

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

Challenge 13

In [32]:
import re
import urllib.parse
from Crypto.Cipher import AES

def routine(input: str):
  result = dict()
  pairs = input.split('&')
  for pair in pairs:
    key, value = pair.split('=')
    result[key] = value
  return result


def profile_for(input: str):
  input = re.sub(r'\&', '', input)
  input = re.sub(r'\=', '', input)
  params = {'email': input, 'uid': 10, 'role': 'user'}
  return urllib.parse.urlencode(params)


In [35]:
block_size = 16
profile = profile_for('foo@bar.com')
print(f'profile: {profile}')
key = random_aes_key()
print(key)
encrypt_profile = encrypt_ECB(str.encode(profile), key, block_size)
print(encrypt_profile)
decrypt_profile = decrypt_ECB(encrypt_profile, key)
print(decrypt_profile)

target = 'email=foo%40bar.com&uid=10&role=admin'
print(f'target: {target}')
encrypt_target = encrypt_ECB(str.encode(target), key, block_size)
print(encrypt_target)
decrypt_target = decrypt_ECB(encrypt_target, key)
print(decrypt_target)

profile: email=foo%40bar.com&uid=10&role=user
b'I\xab\xc5\xfa\xacZ+\x94\xf0\xdaYH\x85\xd4\xb7\xed'
b'\xf6\x8d\\,+\xb0\xb1=v\xb3R<\xe1\xdf\xe83\n0\x0f\xe9A+\x86T"\xd6\xad\xa9\x1a\xaa\ndF\xd2\x10svJ\x8e\xd8\xe4\xc3T]\xff\xa2\xa3Q'
b'email=foo%40bar.com&uid=10&role=user\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
target: email=foo%40bar.com&uid=10&role=admin
b'\xf6\x8d\\,+\xb0\xb1=v\xb3R<\xe1\xdf\xe83\n0\x0f\xe9A+\x86T"\xd6\xad\xa9\x1a\xaa\nd\xc5\xe63NO\x13\'\xccVc{,\xf1Q\xf9\x00'
b'email=foo%40bar.com&uid=10&role=admin\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [36]:
origin = profile_for('foo@bar.com')
print([origin[i*16:(i+1)*16] for i in range(5)])
encrypt_origin = encrypt_ECB(str.encode(origin), key, block_size)
print([encrypt_origin[i*16:(i+1)*16] for i in range(5)])
hack = profile_for('foo@bar.admin')
print([hack[i*16:(i+1)*16] for i in range(5)])
encrypt_hack = encrypt_ECB(str.encode(hack), key, block_size)
print([encrypt_hack[i*16:(i+1)*16] for i in range(5)])

['email=foo%40bar.', 'com&uid=10&role=', 'user', '', '']
[b'\xf6\x8d\\,+\xb0\xb1=v\xb3R<\xe1\xdf\xe83', b'\n0\x0f\xe9A+\x86T"\xd6\xad\xa9\x1a\xaa\nd', b'F\xd2\x10svJ\x8e\xd8\xe4\xc3T]\xff\xa2\xa3Q', b'', b'']
['email=foo%40bar.', 'admin&uid=10&rol', 'e=user', '', '']
[b'\xf6\x8d\\,+\xb0\xb1=v\xb3R<\xe1\xdf\xe83', b'+B\xfa\xc6\xe5\xfe\xf5u\xb9\x7f\xe2A <^\xeb', b'X9h6\x88~\xae-R\xfc\xd8D\xb163\xb3', b'', b'']


In [38]:
encrypt_origin = encrypt_origin.replace(encrypt_origin[32:48], encrypt_hack[16:32]) + encrypt_hack[32:48]
print(encrypt_origin)
print(decrypt_ECB(encrypt_origin, key))

b'\xf6\x8d\\,+\xb0\xb1=v\xb3R<\xe1\xdf\xe83\n0\x0f\xe9A+\x86T"\xd6\xad\xa9\x1a\xaa\nd+B\xfa\xc6\xe5\xfe\xf5u\xb9\x7f\xe2A <^\xebX9h6\x88~\xae-R\xfc\xd8D\xb163\xb3X9h6\x88~\xae-R\xfc\xd8D\xb163\xb3'
b'email=foo%40bar.com&uid=10&role=admin&uid=10&role=user\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00e=user\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [39]:
hack = profile_for('foo@bar.admin')
print([hack[i*16:(i+1)*16] for i in range(5)])
encrypt_hack = encrypt_ECB(str.encode(hack), key, block_size)
print([encrypt_hack[i*16:(i+1)*16] for i in range(5)])

['email=foo%40bar.', 'admin&uid=10&rol', 'e=user', '', '']
[b'\xf6\x8d\\,+\xb0\xb1=v\xb3R<\xe1\xdf\xe83', b'+B\xfa\xc6\xe5\xfe\xf5u\xb9\x7f\xe2A <^\xeb', b'X9h6\x88~\xae-R\xfc\xd8D\xb163\xb3', b'', b'']


Challenge 14

In [145]:
random_prefix = os.urandom(random.randint(1, 32))
print(random_prefix, len(random_prefix))
print(random_key)

b'\x13\xc5' 2
b'}vp~\x1cC\\!M\xce\x80\xd96#\xc1\xd2'


In [46]:
attacker_controlled = b'attacker_controlled'
encrypted_btyes = new_oracle(random_prefix + attacker_controlled)
print(encrypted_btyes)

b'\xa1.md\x80h\xfd\xe32\xee\x96^n\xef\xc5\x87\xcc\xef\xbeUp\xa7S\x10f$\xdb1\xde\xd18u\xa4\xda\xf1\x8ev9H\xc2V\x9ag\x00L\xbaRkY\x19\x05Pr\xc1\x95\xfcZ?]n\xc3\x86\x9c.-;=\xa8o\xf1\x1a\xce\xe6D63KV\\\xbc\x19\xe8\xd9\x05}\x94\x1c\xa0\x17l\x9dOM\xa7\x87?E\xb5o\xd8\x88a\xc5\xbb\xef\xbcc\x8b\x0c4\xcdYWF\x1e\xd3KM9\xbf#\x06\xcf\xb0\xb4M\x02\xc3\x9f\xba\x16,\x9d+\xdb\xc0\xfb\xccV_\x9d\xa2;\xa9H\xcd&?\xaa\xd4\xe2\x0c\x93)\x12\xddy\x08\x88\xa3_6,\xf0\x91\xda\nn\xc3\x0e[}\r\xc7\xdf\xa5\x99k\xa3\xc4\x05\xa2\xbf\xfc,\xc6\x8c~\xefx\xcd\xea'


In [141]:
def get_prefix_length():
  encrypted_btyes_list = []
  for i in range(32):
    encrypted_btyes = new_oracle(b'A' * i)
    print(i, [encrypted_btyes[i*16: (i+1)*16] for i in range(len(encrypted_btyes) // 16 + 1)])
    encrypted_btyes_list.append(encrypted_btyes)

  same_btyes_index = -1
  for j in range(len(encrypted_btyes_list[0])):
    if same_btyes_index != -1:
      break
    for i in range(len(encrypted_btyes_list) - 1):
      #print(encrypted_btyes_list[i][j*block_size: (j+1)*block_size])
      if same_btyes_index == -1:
        if encrypted_btyes_list[i][j*block_size: (j+1)*block_size] == encrypted_btyes_list[i + 1][j*block_size: (j+1)*block_size]:
          if i == 0:
            break
          same_btyes_index = i
  return j, same_btyes_index

def craft_input_dict_14(header: str, prefix_block, prefix_size):
  input_dict = {new_oracle(b'A' * prefix_size + header + bytes([i]))[prefix_block*block_size:(prefix_block+1)*block_size]: bytes([i]) for i in range(256)}
  return input_dict

def match_output_14():
  header = b'A' * (block_size - 1)
  result = header
  prefix_block, prefix_size = get_prefix_length()
  print(prefix_block, prefix_size)

  for x in range(10):
    for i in range(block_size):
      input_dict = craft_input_dict_14(header, prefix_block, prefix_size)
      key = new_oracle(b'A' * prefix_size + b'A' * (block_size - 1 - i))[(x+prefix_block)*block_size:(x+prefix_block+1)*block_size]
      #print(input_dict)
      #print(key)
      if key in input_dict:
        output = input_dict[key]
      else:
        return result[block_size - 1:]
      result += output
      header = result[-block_size+1:]
  return result[block_size - 1:]

In [137]:
match_output_14()

0 [b'\xa1.md\x80h\xfd\xe32\xee\x96^n\xef\xc5\x87', b'\xd1m\xba]\xd2\x97l\xa2\xbfaO\xcff\r\xc0\r', b'\xeb#eJ\x1c.\xcf\xcd\x03\xce\xdf_\xe8\xf1\xa0\xea', b'\x1c\xb2W\xd9\xa2\xc5M\x9f\x8dM\xaf\xdc\x83\x9a\x95\x1c', b'zq\x90\x8f\xe8\x8c\xc2\xdb\xdc\xbb\xa3\nCVvv', b'D\xeb1N\xb9\xc4PLO\x9bN\xb3Q\xe0\xe8\x14', b'\r\tw\x9b\xb7\xfdi\xdet\xb3J\x10:$^Z', b'<\x88){B\xb2\xaa\xb0\xe0\\\xe6Y(\xf1<d', b'o\x83\xd7\xc3\xf3;\x9f\x02\xb43\xf5\xddC\xd5\x1d\xb8', b'\xb4\x14U\xdb\x99\xaf"\x00\xc4x{B\xb3\x9f\x86\xbc', b'*\x13D\xab\xe9\x1d\x17\xcas\xf4\x07OI\x08\xc9\xcd', b'']
1 [b'\xa1.md\x80h\xfd\xe32\xee\x96^n\xef\xc5\x87', b"\x8d\x1a\x7f\x9e\xcb'S~}\x80V\x05\xdf\x99=\x10", b'?_\xdd\x971q\xadag>\x97]L\x0c\xea\xe5', b'\x06\xfbF*\xdewnj%\x98AX\xbf\xfc\xca\xef', b'! \x83%\xfd\xe6x\x83K\x94c\xd3\xd6\x0b\x8c\xf2', b'\xb5\xbf\xe2\xa6\nCW\xdb\xfd\xc1\xe5\xf4\xf29\x82\xdb', b'\x1e\xf0\x82N\xd1\x88;\xfe\xbc\xc1\xcc%\xb3\xa9T\xe7', b'\xbei,xME$\x82\xa5\xbf\x0bE\xce}\xe7\xd7', b'd\xb7\xb3\xd5z\xb1\r\xd7\x8c\x19\xc9j\

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

In [118]:
for i in range(5):
  output = new_oracle(b'A' * 5 + b'A'*15 + bytes([i]))
  print([output[x*block_size: (x+1)*block_size] for x in range(len(output))], bytes([i]))

new_oracle [b'\xee\x18q%t\xca~;\xf5\xb0\xa6\x1eKRi\xa3', b'\x95K\xa2\tG\xbf\x9fl\xf5TeAAAAA', b'AAAAAAAAAAAAAAA\x00', b"Rollin' in my 5.", b'0\nWith my rag-to', b'p down so my hai', b'r can blow\nThe g', b'irlies on standb', b'y waving just to', b' say hi\nDid you ', b'stop? No, I just', b' drove by\n', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', 

In [142]:
get_prefix_length()

0 [b'\x97p\x05e\xea:\nc\xf7\xf8>\xd13?;\x05', b'\xb6\xfe\xf3\x11\xac[\x19\xb4,\x14,\x9b\n\xad\xda\xea', b'\x9b \x10\xe82\xe5U+S\xe8[\xeby\xf6\xe4=', b"\x97\x10\x11';\x19\x98\xd3\xc6\x8a\xb6\xb6\xe7'\x07{", b'wr\xceh\x84g\xe2\x01\x03\xa6\xd9I*5 \x1a', b'\x1dbpbV\xdb\x14\x11\\fU\xa5\x08\xd4\xda\xd7', b'\xcc~\x11\x1a\x84\t8\xcdI\x98\\\xe11\xac\xbd?', b'\xa3\xf6;\xb2\xa8\xcf,\r4\x94B~q\xc0\x8fF', b'\x11P\xe5A\xd3n\x8c\tNI\x0b[\xb3\xe8\xc4\xb7', b'']
1 [b'\x13e^z\x88\x97\xfcB\x9bf*~6\x06\xde\xa3', b'\xd4Eos\x93\xe8\x8a\xfa\xd8h6\x12\x88\x14PV', b'n\xdc\xcd\xf4\x01\xdb\xa9\x9a\xd7?\x1e"Hc\x04s', b'Z{\xdd\t\x1bJ\x87\xb8\xe6.Eh\xe3\x8e?\x99', b'p\xd6ZZS\xbf\x82\xcevk\xc4\x10\x17F\x0fo', b'|\xc1\r\xc6KRF\xd6\x91>\xfc\x91\xd7]\r\x1d', b'\xd26t\xbc\xfd\x8b[\xba\xae\x01\x1aL9\x8d\xb4&', b'5\x18n\x15\xef\xe7\x8c"\x12q\x7fx8*^2', b'\x99\xce;\xa3\xff\xd6\xb5\x7fp#\xad<\xc7\xa1\xf8\x96', b'']
2 [b"'\xdb\x86\xfeO\xeb-\xeb\xec\x86)6]\xe9\x06b", b'\xd7T\x8a\xd28\xff\xe9\xb5\xa3\xdfx=\xa8\xe7\xa5!', b"LDP

(1, 11)